In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec

/home/boffskbob/dev/rate-mate/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# pinecone index is basically a mini db in the pinecone instance
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=768, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [ ]:
import json
data = json.load(open("reviews.json"))
data['reviews']

In [11]:
processed_data = []
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("text-embedding-004")
for review in data['reviews']:
    response = genai.embed_content(model="models/text-embedding-004", content=review['review'])
    embedding = response['embedding']
    processed_data.append({
        "values" : embedding,
        "id" : review['professor'],
        "metadata" : {
            "review" : review['review'],
            "university" : review['university'],
            "stars" : review['stars']
        }
    })
# gemini time

In [14]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace='ns1'
)

{'upserted_count': 20}